In [ ]:
'''
talib
'''

In [ ]:
import talib
import numpy as np

In [ ]:
high = np.array([1.0, 2.0, 6.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
low = np.array([1.0, 2.0, 5.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0])
close = np.array([1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
a = talib.ATR(high, low, close, 2)
a

In [ ]:
close = np.array([1000.0, 2.0, 1.0, 6.0, 6.0, 1.0, 6.0, 1.0, 1.0, 1.0, 1.0])
result = talib.RSI(close, 2)
result

In [ ]:
'''
tushare
'''

In [15]:
import tushare as ts
from datetime import datetime, timedelta

In [16]:
df = ts.get_tick_data('rb1806', '2017-06-16')
df

,time,price,change,volume,amount,type
0,"alert(""当天没有数据"");",NaN,NaN,NaN,NaN,NaN
1,window.close();,NaN,NaN,NaN,NaN,NaN
2,</script>,NaN,NaN,NaN,NaN,NaN


In [17]:
cons = ts.get_apis()

In [28]:
df = ts.tick('rb1805', conn = cons, date = '2017-12-21', asset = 'X') 
#df = ts.tick('IF1801', conn = cons, date = '2017-06-22', asset = 'X') 
#df = ts.tick('600380', conn = cons, date = '2018-01-02')
#df = ts.tick('rb1806', conn = cons, date = '2018-01-10', asset = 'X')
df

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()


In [1]:
lastDt = None
mSecond = 0
for i, row in df.iterrows():
    dt = row['date'].to_datetime()
    if (not lastDt) or lastDt != dt:
        mSecond = 0
        lastDt = dt
    else:
        mSecond += 1
    
    #print mSecond
    dt = dt.replace(microsecond = mSecond)
        
    #print dt, row['vol']

NameError: name 'df' is not defined

In [ ]:
for i, row in df.iterrows():
    print row['date'], '*'*2, row['vol']

In [ ]:
df = ts.bar('rb1805', conn = cons, freq = 'D', start_date = '2017-01-01', end_date = '', asset = 'X')

In [ ]:
df.sort_index(inplace = True)
df

In [ ]:
a = df.ix[1]
the = a['date']
dt = the.to_datetime()
print dt
if dt.time() >= datetime.time(21, 0):
    dt = dt - timedelta(1)

print dt

In [ ]:
for index, row in df.iterrows():
    print row['date'], ' '*2, row['vol']

In [ ]:
from datetime import datetime

In [ ]:
from collections import namedtuple
abc = namedtuple('xyz', ['x', 'y'])
the = abc(6,2)
the.x

In [ ]:
import sys
#sys.getdefaultencoding()
#reload(sys)
#sys.setdefaultencoding('UTF-8')
a = sys.getdefaultencoding()
type(a)

In [ ]:
if isinstance('a', str):
    print 1
else:
    print 2

In [ ]:
import pymongo 

In [ ]:
c = pymongo.MongoClient('localhot', 270)
if c:
    print c
    
else:
    print 'no'

In [ ]:
'''
datetime
'''

In [ ]:
from datetime import datetime, time

In [ ]:
a = datetime.now().time()
print type(a)

In [ ]:
a = False
if not a:
    print 'yes'

In [ ]:
'''
json
'''
import json

In [ ]:
f1 = open(filePath)
the = json.load(f1)
for dic in the:
    print dic['a']

In [ ]:
import os

In [ ]:
currentFolder = os.getcwd()
print currentFolder
newPath = os.path.join(currentFolder, '')

In [31]:
d = datetime.now().strftime('Y-m-d')
d

'Y-m-d'

In [ ]:
'''
pandas
'''

In [49]:
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [64]:
b = DataFrame()
b['名称'] = []
b['价格'] = []
b['开盘价'] = []
b.ix['2017-01-01'] = [1,2,3]
b.ix['2017-01-02'] = [1,2,3]
b.ix['2017-01-03'] = [1,2,3]
b

,名称,价格,开盘价
2017-01-01,1.0,2.0,3.0
2017-01-02,1.0,2.0,3.0
2017-01-03,1.0,2.0,3.0


In [65]:
fileName = 'abc.csv'

In [66]:
b.to_csv(fileName, encoding = 'utf-8')

In [67]:
c = pd.read_csv(fileName, encoding = 'utf-8')
c

,Unnamed: 0,名称,价格,开盘价
0,2017-01-01,1.0,2.0,3.0
1,2017-01-02,1.0,2.0,3.0
2,2017-01-03,1.0,2.0,3.0


In [ ]:
'''
csv
'''

In [37]:
import csv
import codecs

In [39]:
fileName = 'abc.csv'

In [54]:
# 写
f = file(fileName, 'wb')
f.write(codecs.BOM_UTF8)

fieldnames = [u'a', 'last_name']
writer = csv.DictWriter(f, fieldnames=fieldnames)

writer.writeheader()
writer.writerow({'a': '和', 'last_name': '啊'})

In [56]:
# 读
with open(fileName) as f:
    reader = csv.DictReader(f)
    the = []
    for row in reader:
        the.append(row)
    print the

[]


In [ ]:
'''
os
'''

In [5]:
import os

In [20]:
exists = os.path.exists(fileName)
print exists
print os.getcwd()
newFileName = os.getcwd() + '\clone\\abc.csv'
print newFileName
print os.path.exists(newFileName)

True
C:\Users\loe\Quant\python-for-investment\jupyter
C:\Users\loe\Quant\python-for-investment\jupyter\clone\abc.csv
True


In [28]:
dic = {'a':1, 'b':2, 'c':3}
print dic.keys()
print type(dic.keys())

['a', 'c', 'b']
<type 'list'>


In [13]:
walkingDic = 'C:\\Users'
for root, subdirs, files in os.walk(walkingDic):
    for sub in subdirs:
        if sub == 'stgEarningCSV':
            print root

C:\Users\Rachel\iCloudDrive\com~apple~Numbers\ctaStrategy
yes


In [14]:
print 'yes'

yes


In [1]:
from datetime import datetime

In [8]:
timeStr = '113000500'
the = datetime.strptime(timeStr, '%H%M%S%f')
time = the.strftime('%H:%M:%S.%f')

dateStr = '20170511'

the = datetime.strptime(dateStr + ' ' + time, '%Y%m%d %H:%M:%S.%f')
print the

2017-05-11 11:30:00.500000
